In [2]:
import requests
import pandas as pd
import json

# config
# API: https://rapidapi.com/apidojo/api/transfermarket
# https://www.linkedin.com/company/rapidapi/

In [10]:
url = "https://transfermarket.p.rapidapi.com/clubs/list-by-competition"

headers = {
	"X-RapidAPI-Key": "<RapidAPI-key>",
	"X-RapidAPI-Host": "transfermarket.p.rapidapi.com"
} 

years = ['2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']

# Retrieve clubs from current season
# L1 - German Bundesliga

In [50]:
querystring = {"id":"L1","domain":"en"}

response = requests.get(url, headers=headers, params=querystring)

s = response.json()

j = json.dumps(s).replace("M'gladbach", "Mgladbach").replace("K'lautern", "Klautern").replace("'", '"')

c = json.loads(j)

clubs = pd.json_normalize(c['clubs']).loc[:, 'id':'name']
clubs.to_csv('../data/clubs.csv', index=False)


# Retrieve the list of players by years

In [ ]:
url = "https://transfermarket.p.rapidapi.com/clubs/get-squad"

clubs = pd.read_csv('../data/clubs.csv')

for y in years:
    print(f'year: {y}')
    players = pd.DataFrame(columns=['id', 'name', 'club'])
    for ind in clubs.index:
        print(f'club: {clubs["id"][ind]}')
        querystring = {"id":clubs['id'][ind], "saison_id":y, "domain":"en"}

        response = requests.get(url, headers=headers, params=querystring)

        s = response.json()

        #print(s)

        j = json.dumps(s).replace("M'gladbach", "Mgladbach").replace("K'lautern", "Klautern").replace("'", '"')

        p = json.loads(j)

        club_players = pd.json_normalize(p['squad']).loc[:, 'id':'name']
        club_players['club'] = clubs['id'][ind]
        
        players = pd.concat([players, club_players])

    players['year'] = y
    players.to_csv(f'../data/players-{y}.csv', index=False)
